Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

## Answer:
The procecutor points out that there is a .01 chance the defendent has the guilty blood type. However, he incorrectly claims that the inverse probablity applies to the prosecutor's guilt since he has the blood type. In reality all that has been proven is that it is *possible* that the defendent is guilty since he lies within the popupation of the universe with the rare blood type. Without any other evidence the correct probability could very well be represented as $\frac{1}{population of the universe with the rare blood type}$.


b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

## Answer:
While the defendent is correct that the probability in this case is 1/8000, the probability is relevant. If other evidence was presented, such as a matching hair color, the probability of the blood type would be combined with the new probabilty, resulting in a higher chance of guilt than the probabilities would suggest seperately.

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

## Defining Variables
#### Bayes Theorem
$P(C|X) = \frac{P(X|C)P(C)}{P(X)}$  
##### Expanded:
$\frac{P(X|C)P(C)}{P(X|C)P(C) + P(X|!C)P(!C)}$


## Q: Is the defendent guilty assuming he tested positive for the blood type *AND* was one of 5 people with access to the crime scene?
$P(G|X)$ = Probabilty the defendent is guilty assuming he tested positive

$P(X|G)$ (*Likelihood*) = .98

$P(G)$ (*Prior*) = Probabity the defendent is the murderer out of 5 other known people $ = \frac{1}{5}$

P(X|!G) = Chance the test comes back positive and he's innocent = 0.01

P(!G) = Chance the defendent is innocent = $\frac{4}{5}$


$P(C|X) =\frac{ .98 * \frac{1}{5}}{(.98 * \frac{1}{5})+(.01 * \frac{4}{5})}$ = **0.96078431372**

### Answer: 0.96078431372

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [1]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [14]:
from __future__ import division
#What is the prior P(Y=ham) ?
ham_prior = np.sum(training_labels)/training_labels.size

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = val[1]/(val[0]+val[1])
    

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [21]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = ham_prior
    posterior_spam = 1 - ham_prior
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler?
        try:
            posterior_ham *= ham_likelihood[w]
            posterior_spam *= (1 - ham_likelihood[w])
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)
    

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [45]:
posts = np.mean(posteriors, axis = 0)
ham_post = posts[1]
spam_post = posts[0]

ham_prob_test = np.sum(test_labels)/test_labels.size

print("AVG POSTERIOR PROBABILITY OF HAM: " + str(ham_post))
print("AVG POSTERIOR PROBABILITY OF SPAM: " + str(spam_post))

print("\nAVG REAL PROB OF HAM: " + str(ham_prob_test))
print("AVG REAL PROB OF SPAM: " + str(1-ham_prob_test))

classified_array = np.where(posteriors[:,1] > .7, 1, 0) # If the message has a 30% chance or more of being spam it will be classified as spam
precentage_array = np.where(classified_array == test_labels, 1, 0)

print("\nPrecentage correct: " + str(np.sum(precentage_array)/precentage_array.size))

AVG POSTERIOR PROBABILITY OF HAM: 0.909737571387
AVG POSTERIOR PROBABILITY OF SPAM: 0.0902624286129

AVG REAL PROB OF HAM: 0.849192100539
AVG REAL PROB OF SPAM: 0.150807899461

Precentage correct: 0.940754039497
